<a href="https://colab.research.google.com/github/SanjayJanardhan-89/ApacheSparkHandsOn/blob/main/ComplexDatatypes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark.sql import SparkSession

spark= SparkSession \
       .builder \
       .appName("OurSparkApp") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,678 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,692 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,763

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col


**Creating Arrays**

In [28]:
df_sql_array = spark.sql("SELECT array('KGF 1', 'KGF 2', 'Autograph', 'Kicha','Hucha') as movies")
df_sql_array.printSchema()
df_sql_array.show(truncate=False)

root
 |-- movies: array (nullable = false)
 |    |-- element: string (containsNull = false)

+---------------------------------------+
|movies                                 |
+---------------------------------------+
|[KGF 1, KGF 2, Autograph, Kicha, Hucha]|
+---------------------------------------+



In [36]:
from pyspark.sql.types import StructType,StructField, StringType, ArrayType,MapType

data = [
          [
            ["KGF 1", "KGF 2", "Autograph", "Kicha","Hucha"]
          ]
        ]

# Schema
schema = StructType([
             StructField('movies', ArrayType(StringType()), True),
     ])

# Create DataFrame
df_array = spark.createDataFrame(data = data, schema = schema)
df_array.printSchema()
df_array.show(truncate=False) # shows all columns

root
 |-- movies: array (nullable = true)
 |    |-- element: string (containsNull = true)

+---------------------------------------+
|movies                                 |
+---------------------------------------+
|[KGF 1, KGF 2, Autograph, Kicha, Hucha]|
+---------------------------------------+



**Creating Map type**

In [35]:
df_sql_map = spark.sql("SELECT map('Building','500 CR', 'Commercal',100) as income")
df_sql_map.printSchema()
df_sql_map.show(truncate=False)

root
 |-- income: map (nullable = false)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = false)

+--------------------------------------+
|income                                |
+--------------------------------------+
|{Building -> 500 CR, Commercal -> 100}|
+--------------------------------------+



In [42]:
from pyspark.sql.types import StructType,StructField, StringType, ArrayType,MapType


data = [

          ({"Bank":"100 CR", "Business":"50 CR", "Land":"150 CR"}),
          # ({"Others":"300 CR"}),
          # ({"Building":"500 CR", "Commercal":100}),
          # ({"Building":"500 CR", "Commercal":100}),

      ]


# Schema
schema = StructType([
               StructField('properties', MapType(StringType(),StringType()), True)
        ])

# Create DataFrame
df_map = spark.createDataFrame(data = data, schema = schema)
df_map.printSchema()
df_map.show(truncate=False) # shows all columns

root
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+---------------------------------------------------+
|properties                                         |
+---------------------------------------------------+
|{Bank -> 100 CR, Land -> 150 CR, Business -> 50 CR}|
+---------------------------------------------------+



In [ ]:
|spark.sql("SELECT struct(1, 2, 3) as ex_struct")

DataFrame[ex_struct: struct<col1:int,col2:int,col3:int>]

In [ ]:
df_struct = spark.sql("SELECT struct(1, 2, '3') as ex_struct")
df_struct.show()

+---------+
|ex_struct|
+---------+
|{1, 2, 3}|
+---------+



In [ ]:
df_struct.select("ex_struct.col3", "ex_struct").show()

+----+---------+
|col3|ex_struct|
+----+---------+
|   3|{1, 2, 3}|
+----+---------+



In [ ]:
df_struct.select("ex_struct.*").show(truncate=False)

+----+----+----+
|col1|col2|col3|
+----+----+----+
|1   |2   |3   |
+----+----+----+



In [ ]:
df_map = spark.sql("SELECT map(1.0, '2', 3.0, '4') as ex_map")
df_map.show()

+--------------------+
|              ex_map|
+--------------------+
|{1.0 -> 2, 3.0 -> 4}|
+--------------------+



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, ArrayType,MapType

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

# Data
data = [
        (("James",None,"Smith"),"OH","M"),
        (("Anna","Rose",""),"NY","F"),
        (("Julia","","Williams"),"OH","F"),
        (("Maria","Anne","Jones"),"NY","M"),
        (("Jen","Mary","Brown"),"NY","M"),
        (("Mike","Mary","Williams"),"OH","M")
        ]

# Schema
schema = StructType([
    StructField('name', StructType([
         StructField('firstname', StringType(), True),
         StructField('middlename', StringType(), True),
         StructField('lastname', StringType(), True)
         ])),
     StructField('state', StringType(), True),
     StructField('gender', StringType(), True)
     ])

# Create DataFrame
df2 = spark.createDataFrame(data = data, schema = schema)
df2.printSchema()
df2.show(truncate=False) # shows all columns

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)

+----------------------+-----+------+
|name                  |state|gender|
+----------------------+-----+------+
|{James, NULL, Smith}  |OH   |M     |
|{Anna, Rose, }        |NY   |F     |
|{Julia, , Williams}   |OH   |F     |
|{Maria, Anne, Jones}  |NY   |M     |
|{Jen, Mary, Brown}    |NY   |M     |
|{Mike, Mary, Williams}|OH   |M     |
+----------------------+-----+------+



In [ ]:
df2.select("name").show()

+--------------------+
|                name|
+--------------------+
|{James, NULL, Smith}|
|      {Anna, Rose, }|
| {Julia, , Williams}|
|{Maria, Anne, Jones}|
|  {Jen, Mary, Brown}|
|{Mike, Mary, Will...|
+--------------------+



In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, ArrayType,MapType
from pyspark.sql.functions import element_at

data = [
        ([("Yash","K",None),("Yash2","K2",None)],"BL","M",["KGF 1", "KGF 2"],{"Bank":"100 CR", "Business":"50 CR", "Land":"150 CR"}),
        ([("Sudeep","Kicha","S")],"DL","M",["Autograph", "Kicha","Hucha"],{"Others":"300 CR"}),
        ([("Puneeth",None,"Raj")],"MB","M",[], {"Building":"500 CR", "Commercal":100}),
        ([("Darshan",None,None)],"MB","M",[], {"Building":"500 CR", "Commercal":100}),

        ]

# Schema
schema = StructType([
    StructField(    'name'
                  , ArrayType(
                        StructType([
                            StructField('firstname', StringType(), True),
                            StructField('middlename', StringType(), True),
                            StructField('lastname', StringType(), True)
                        ])
                    ), True),
     StructField('state', StringType(), True),
     StructField('gender', StringType(), True),
     StructField('movies', ArrayType(StringType()), True),
     StructField('properties', MapType(StringType(),StringType()), True)
     ])

# Create DataFrame
df2 = spark.createDataFrame(data = data, schema = schema)
df2.printSchema()
df2.show(truncate=False) # shows all columns

root
 |-- name: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- firstname: string (nullable = true)
 |    |    |-- middlename: string (nullable = true)
 |    |    |-- lastname: string (nullable = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- movies: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+------------------------------------+-----+------+-------------------------+---------------------------------------------------+
|name                                |state|gender|movies                   |properties                                         |
+------------------------------------+-----+------+-------------------------+---------------------------------------------------+
|[{Yash, K, NULL}, {Yash2, K2, NULL}]|BL   |M     |[KGF 1, KGF 2]           |{Bank -> 100 CR, La

In [ ]:
 (df2
  .select("properties", "name", "name.firstname")
  .withColumn("M-Building", col("properties").getItem("Building"))
  .withColumn("M-Commercal ", col("properties").getItem("Commercal"))
  .withColumn("M-Others ", col("properties").getItem("Others"))
  .withColumn("S-FName", col("name.firstname"))
  ).show()

+--------------------+--------------------+---------+----------+------------+---------+-------+
|          properties|                name|firstname|M-Building|M-Commercal |M-Others |S-FName|
+--------------------+--------------------+---------+----------+------------+---------+-------+
|{Bank -> 100 CR, ...|     {Yash, K, NULL}|     Yash|      NULL|        NULL|     NULL|   Yash|
|  {Others -> 300 CR}|  {Sudeep, Kicha, S}|   Sudeep|      NULL|        NULL|   300 CR| Sudeep|
|{Building -> 500 ...|{Puneeth, NULL, Raj}|  Puneeth|    500 CR|         100|     NULL|Puneeth|
|{Building -> 500 ...|{Darshan, NULL, N...|  Darshan|    500 CR|         100|     NULL|Darshan|
+--------------------+--------------------+---------+----------+------------+---------+-------+



In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, ArrayType,MapType
from pyspark.sql.functions import element_at

data = [
        (("Yash","K",None),"BL","M",["KGF 1", "KGF 2"],{"Bank":"100 CR", "Business":"50 CR", "Land":"150 CR"}),
        (("Sudeep","Kicha","S"),"DL","M",["Autograph", "Kicha","Hucha"],{"Others":"300 CR"}),
        (("Puneeth",None,"Raj"),"MB","M",[], {"Building":"500 CR", "Commercal":100}),
        (("Darshan",None,None),"MB","M",[], {"Building":"500 CR", "Commercal":100}),

        ]

# Schema
schema = StructType([
    StructField('name', StructType([
         StructField('firstname', StringType(), True),
         StructField('middlename', StringType(), True),
         StructField('lastname', StringType(), True)
         ])),
     StructField('state', StringType(), True),
     StructField('gender', StringType(), True),
     StructField('movies', ArrayType(StringType()), True),
     StructField('properties', MapType(StringType(),StringType()), True)
     ])

# Create DataFrame
df2 = spark.createDataFrame(data = data, schema = schema)
df2.printSchema()
df2.show(truncate=False) # shows all columns

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- movies: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+---------------------+-----+------+-------------------------+---------------------------------------------------+
|name                 |state|gender|movies                   |properties                                         |
+---------------------+-----+------+-------------------------+---------------------------------------------------+
|{Yash, K, NULL}      |BL   |M     |[KGF 1, KGF 2]           |{Bank -> 100 CR, Land -> 150 CR, Business -> 50 CR}|
|{Sudeep, Kicha, S}   |DL   |M     |[Autograph, Kicha, Hucha]|{Others -> 300 CR}        

In [4]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
  StructField("DEST_COUNTRY_NAME", StringType(), True),
  StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
  StructField("count", LongType(), False, metadata={"hello":"world"})
])
df = spark.read.format("json").schema(myManualSchema)\
  .load("sample_data/2015-summary.json")

In [5]:
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [6]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [12]:
spark.read.format("json").load("sample_data/2015-summary.json").schema

StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', LongType(), True)])

In [10]:
df2.pr

AttributeError: 'StructType' object has no attribute 'printSchema'